레이블 데이터 만들기

In [4]:
#패키지 불러오기

# import matplotlib
# matplotlib.use('Agg')
# import glob
# import argparse

# from shutil import copyfile, move
# from pathlib import Path
import pandas as pd
import matplotlib.dates as mdates

In [15]:
def createLabel(fname, seq_len):
    print("Creating label . . .")

    filename = fname.split('_')
    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    df.fillna(0)##빈칸없음 어차피
    df.reset_index(inplace=True)##인덱스 의 문자열(날짜를)을 숫자날짜(?)로 바꿔줌
    df['Open_time'] = df['Open_time'].map(mdates.date2num)

    for i in range(0, len(df)):
        c = df.iloc[i+19:i+19+ int(seq_len), :]## 19는 우리가 필요한 첫번째 데이터가 첫 그림의 마지막 봉의 클로즈 데이터이므로

        starting = 0
        endvalue = 0
        label = ""

        if len(c.index) == int(seq_len): # 총 데이터가 봉개수(여기선 3 과 같다면)
            starting = c["Open"].iloc[0]#첫번째 봉의 오픈
            endvalue = c["Close"].iloc[-1] # 3번째 봉의 클로즈

            tmp_rtn = endvalue / starting -1
            if tmp_rtn >= 0.007 :
                label = 1 # 매수
            elif 0.007 > tmp_rtn >= 0 : #유지
                label = 2
            else: #매도
                label = 3

            with open("{}_label_{}.txt".format(filename[0][:3], seq_len), 'a') as the_file:
                the_file.write("{}-{},{}".format(filename[0][:3], i, label))
                the_file.write("\n")

    print("Create label finished.")
createLabel("btc_20220113.csv",3)#3은 15분 뒤 데이터라는 뜻

Creating label . . .
Create label finished.


csv 파일을 이미지로 만들기

In [11]:
from mplfinance import original_flavor as mpl
import matplotlib.pyplot as plt
import os

def ohlc2cs(fname, bongsu):
    print("Converting ohlc to candlestick")
    symbol = fname.split('_')[0] # 여기선 btc

    print(symbol)
    path = "{}".format(os.getcwd())

    if not os.path.exists("{}/dataset/{}_{}".format(path,symbol, bongsu)):
        os.makedirs("{}/dataset/{}_{}".format(path,symbol, bongsu))
    df = pd.read_csv(fname, parse_dates=True, index_col=0)
    df.fillna(0)
    plt.style.use('dark_background')
    df.reset_index(inplace=True)
    df['Open_time'] = df['Open_time'].map(mdates.date2num)


    for i in range(0, len(df)-int(bongsu)):

        c = df.iloc[i:i + int(bongsu) - 1, :]


        fig = plt.figure(figsize=(50 / 96,
                                  50 / 96), dpi=96)
        ax1 = fig.add_subplot(1, 1, 1)
        mpl.candlestick2_ochl(ax1, c['Open'], c['Close'], c['High'],c['Low'],
                  width=1,colorup='#77d879', colordown='#db3f3f')
        ax1.grid(False)
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        ax1.xaxis.set_visible(False)
        ax1.yaxis.set_visible(False)
        ax1.axis('off')

        pngfile = "{}/dataset/{}_{}/{}-{}.png".format(path,symbol, bongsu, fname[:3], i)
        fig.savefig(pngfile, pad_inches=0, transparent=False)
        plt.close(fig)

        # Alpha 채널 없애기 위한.
        from PIL import Image
        img = Image.open(pngfile)
        img = img.convert('RGB')
        img.save(pngfile)


    print("Converting olhc to candlestick finished.")
#ohlc2cs("btc_20220113.csv",20)

Converting ohlc to candlestick
btc
Converting olhc to candlestik finished.


이미지 이름에 레이블을 넣기

In [30]:
#1,2,3 폴더로 일단 구분하자
from shutil import copyfile, move

def labeling_img(input, label_file):
    path = "{}/{}".format(os.getcwd(), input)
    label_dict = {}
    with open(label_file) as label_txt:
        for line in label_txt:
            (key, val) = line.split(',')
            label_dict[key] = val.rstrip() #파일명

    for file in os.listdir(path):
        f, e = os.path.splitext(file) ## f  가 파일명이 맞는데 이게 1이면 1폴더로 가고
        if label_dict[f] == "1":
                os.rename("{}/{}".format(path, file),"{}/{}".format(path, file+ "_1.png"))
        elif label_dict[f] == "2":
                os.rename("{}/{}".format(path, file),"{}/{}".format(path, file+ "_2.png"))
        elif label_dict[f] == "3":
            os.rename("{}/{}".format(path, file),"{}/{}".format(path, file+ "_3.png"))

    print('Done')

#labeling_img("/dataset/btc_20", "btc_label_3.txt")

# btc-이미지번호 _ 라벨 .png 형태로 저장됨
# 이후 구글 코랩에서 진행함.

Done
